In [ ]:
import torch
print(torch.__version__)

2.6.0


In [3]:
# === Load Libraries ===
import os
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

# === Set Up Project Path ===
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils.robot_arm import RobotArm

# === Robot Arm Setup ===
num_joints = 3
joint_limits = [(0, 2*np.pi)] * num_joints
rotation_axes = ['z', 'y', 'y']
link_lengths = [1.0] * (num_joints + 1)
link_axes = ['z', 'x', 'x', 'x']

robot_arm = RobotArm(num_joints=num_joints,
                     joint_limits=joint_limits,
                     link_lengths=link_lengths,
                     rotation_axes=rotation_axes,
                     link_axes=link_axes)

# === Load Precomputed FK Data ===
print("Loading FK dataset...")
joint_samples = np.load("joint_samples.npy")
position_samples = np.load("position_samples.npy")
quaternion_samples = np.load("quaternion_samples.npy")

print("Joint samples:", joint_samples.shape)
print("Position samples:", position_samples.shape)
print("Quaternion samples:", quaternion_samples.shape)

# === Build GNN Dataset ===
def build_gnn_data(joint_samples, position_samples, quaternion_samples):
    data_list = []
    for i in range(len(joint_samples)):
        pose_7d = np.concatenate([position_samples[i], quaternion_samples[i]])  # shape (7,)
        q = joint_samples[i]  # target

        x = torch.tensor([pose_7d, pose_7d, pose_7d], dtype=torch.float)  # shape (3, 7)
        y = torch.tensor(np.tile(q, (3, 1)), dtype=torch.float)  # shape (3, 3)
        edge_index = torch.tensor([[0, 1], [1, 2]], dtype=torch.long).t()

        data_list.append(Data(x=x, edge_index=edge_index, y=y))
    return data_list

dataset = build_gnn_data(joint_samples, position_samples, quaternion_samples)

# === GNN Model ===
class IK_GNN(nn.Module):
    def __init__(self, input_dim=7, hidden_dim=32, output_dim=3):
        super(IK_GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

model = IK_GNN()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# === Training Loop ===
def train_gnn(model, dataset, epochs=30):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for data in dataset:
            optimizer.zero_grad()
            out = model(data.x, data.edge_index)
            loss = criterion(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch}, Loss: {total_loss / len(dataset):.6f}")

train_gnn(model, dataset)

# === Evaluation ===
model.eval()
angle_errors = []
ee_errors = []

for i in range(len(dataset)):
    data = dataset[i]
    true_q = data.y[0].numpy()
    with torch.no_grad():
        pred_q = model(data.x, data.edge_index)[0].numpy()

    angle_errors.append(np.abs(pred_q - true_q))

    fk_pred = robot_arm.forward_kinematics(pred_q)[0]
    fk_true = robot_arm.forward_kinematics(true_q)[0]
    ee_errors.append(np.linalg.norm(fk_pred - fk_true))

angle_errors = np.array(angle_errors)
ee_errors = np.array(ee_errors)

print(f"\n=== GNN Evaluation ===")
print(f"Mean Joint Angle Error (radians): {np.mean(angle_errors):.4f}")
print(f"Mean End-Effector Position Error: {np.mean(ee_errors):.4f}")


ModuleNotFoundError: No module named 'torch_geometric'